#  Выпускной SQL-проект

## Цель исследования:

Проанализировать базу данных сервиса для чтения книг


## Задачи:

1. Подготовить и выгрузить необходимые данные
2. Исследовать данные, вывести первые строки
3. Посчитать, сколько книг вышло после 1 января 2000 года
4. Для каждой книги посчитать количество обзоров и среднюю оценку
5. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (так как мы исключаем из анализа брошюры)
6. Определить автора с самой высокой средней оценкой книг (для учёта только книг с 50 и более оценками)
7. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок
8. Написать выводы по каждой задаче

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подготовка и выгрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
from sqlalchemy import create_engine, text

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})



## Исследование данных

**Рассмотрим таблицу books**



In [21]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

books = pd.io.sql.read_sql(sql=text(query), con = con)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Рассмотрим таблицу authors**

In [22]:
query = '''SELECT * FROM authors'''

authors = pd.io.sql.read_sql(sql=text(query), con = con)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Рассмотрим таблицу publishers**

In [23]:
query = '''SELECT * FROM publishers'''

publishers = pd.io.sql.read_sql(sql=text(query), con = con)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Рассмотрим таблицу ratings**

In [24]:
query = '''SELECT * FROM ratings'''

ratings = pd.io.sql.read_sql(sql=text(query), con = con)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Рассмотрим таблицу reviews**

In [25]:
query = '''SELECT * FROM reviews'''

reviews = pd.io.sql.read_sql(sql=text(query), con = con)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<div style="border:solid green 3px; padding: 20px">

#### *Посмотрели данные по таблицам и вывели первые 5 строк каждой*

### Задача 1. Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
           SELECT COUNT(book_id)
           FROM books
           WHERE publication_date >= '2000-01-01'
           
        '''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,821


<div style="border:solid green 3px; padding: 20px">

#### *После 1 января 2000 года вышло 821 книги*

### Задача 2. Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''
           SELECT b.title, COUNT (DISTINCT rev.review_id) as count_review, ROUND (avg(r.rating), 2) as rating
           FROM reviews AS rev
           RIGHT JOIN books AS b ON rev.book_id = b.book_id
           LEFT JOIN ratings AS r ON b.book_id = r.book_id
           GROUP BY b.book_id
           ORDER BY rating DESC

        '''

pd.io.sql.read_sql(query, con = engine)

,title,count_review,rating
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,Light in August,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,His Excellency: George Washington,2,2.00
997,Drowning Ruth,3,2.00
998,Junky,2,2.00


<div style="border:solid green 3px; padding: 20px">
    
#### *Для каждой книги посчитали среднюю оценку и кол-во обзоров*

### Задача 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц (для исключения из анализа брошюр)

In [9]:
query = '''
            SELECT 
            b.publisher_id, 
            p.publisher, 
            COUNT(*) as count_books
            FROM books as b
            LEFT JOIN publishers as p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY b.publisher_id, p.publisher
            ORDER BY count_books DESC
            LIMIT 1
            
        '''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


<div style="border:solid green 3px; padding: 20px">

#### *Выявили издательство, которое выпустило наибольшее число книг толще 50 стр это  Penguin Books, количество выпущенных книг 42*

### Задача 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query = '''
           SELECT
           a.author_id as author_id,
           a.author as author,
           AVG(r.rating) AS avg_rating,
           COUNT(r.rating) AS count_rating
           FROM authors as a
           INNER JOIN books ON books.author_id = a.author_id
           INNER JOIN ratings as r ON r.book_id = books.book_id
           GROUP BY a.author_id, a.author
           HAVING COUNT(r.rating) >= 50 
           ORDER BY avg_rating DESC
           LIMIT 1
           
        '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg_rating,count_rating
0,130,Diana Gabaldon,4.3,50


<div style="border:solid green 3px; padding: 20px">
    
#### *Автор с самой высокой средней оценкой книг это Diana Gabaldon. Средний рейтинг 4.3, кол-во оценок 50*

### Задача 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = ''' 
           SELECT
           AVG(count_reviews) AS avg_reviews 
           FROM 
           (SELECT username, COUNT(review_id) AS count_reviews
           FROM reviews
           WHERE username IN 
           (SELECT username
           FROM ratings
           GROUP BY username
           HAVING COUNT(rating_id) > 48)
           GROUP BY username ) AS rev
           
        ''' 


pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.0


<div style="border:solid green 3px; padding: 20px">
    
#### *Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок это 24*

<div style="border:solid green 3px; padding: 20px">
    
## Общий вывод
    
**В ходе исследования мы выявили следующее:**
    
    1. После 1 января 2000 года вышло 821 книги
    2. Издательство, которое выпустило наибольшее число книг толще 50 стр это Penguin Books, количество выпущенных книг 42
    3. Автор с самой высокой средней оценкой книг это Diana Gabaldon. Средний рейтинг 4.3, кол-во оценок 50
    4. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок это 24